In [91]:
import numpy as np
from numpy.linalg import qr
from scipy.linalg import norm

def get_mat(file):
    mat = []
    answ = []
    with open(file) as file_handler:
        for line in file_handler:
            row = list(map(int, line.split(',')[:-1]))
            mat.append(row)
    return np.array(mat)


def arnoldi_iteration(A, b, n: int):
    m = A.shape[0]
    h = np.zeros((n + 1, n))
    Q = np.zeros((m, n + 1))
    q = b / np.linalg.norm(b) 
    Q[:, 0] = q  
    for k in range(n):
        v = A.dot(q) 
        for j in range(k + 1):  
            h[j, k] = np.dot(Q[:, j].conj(), v)
            v = v - h[j, k] * Q[:, j]

        h[k + 1, k] = np.linalg.norm(v)
        eps = 0.001  
        if h[k + 1, k] > eps:  
            q = v / h[k + 1, k] 
            Q[:, k + 1] = q
        else:  
            return Q[:,0:n], h[0:n,:]
    return Q[:,0:n], h[0:n,:]


def get_mat(file):
    mat = []
    answ = []
    with open(file) as file_handler:
        for line in file_handler:
            row = list(map(int, line.split(',')[:-1]))
            mat.append(row)
    return np.array(mat)

def get_v(column, size, k):
    v = np.zeros(size)
    v[k] = column[k]+np.sign(column[k])*np.sqrt(np.sum(column[k:]*column[k:]))
    for i in range(k+1, size):
        v[i] = column[i]
    return v
        

def get_H(column, size, k):
    v = get_v(column,size,k)[:, np.newaxis]
    return np.eye(size) - 2*np.dot(v,v.T)/np.dot(v.T,v)
    

def get_A(H,A0):
    return np.dot(H,A0)

def get_QR(A):
    size = A[:,1].size
    Q = np.eye(size)
    for i in range(size-1):
        H = get_H(A[:,i],A[:,i].size,i)
        A = get_A(H,A)
        Q = np.dot(Q,H)
    return Q,A

def solve_equation(A, i):
    size = A[:,1].size
    if i+1 < size:
        a12 = A[i][i + 1]
        a21 = A[i + 1][i]
        a22 = A[i + 1][i + 1]
    else:
        a12 = 0
        a21 = 0
        a22 = 0
    a11 = A[i][i]
    a = 1
    b = -a11 - a22
    c = a11 * a22 - a12 * a21
    D = b*b-4*a*c
    if D < 0:
        l1 = (-b+np.sqrt(complex(D)))/2
        l2 = (-b-np.sqrt(complex(D)))/2
    else:
        l1 = (-b+np.sqrt(D))/2
        l2 = (-b-np.sqrt(D))/2
    return np.array([l1,l2])


def special_qr(A, eps=1E-14, maxiter=5000):
    m = A.shape[0]
    Q = np.identity(m)
    residual = 10
    lprev = np.ones(m)
    ctr = 0
    while norm(residual) > eps:
        Q,R = qr(A@Q)
        lam = np.diagonal(Q.T @ A @ Q) #Матрица коэффициентов Рэлея
        #print(lam)
        residual = norm(lprev - np.sort(lam))
        lprev = np.sort(lam)
        ctr += 1
        if ctr == maxiter: break
    #print(ctr)
    print("Матрица собственных векторов")
    for i in range(1,len(Q)):
    	Q[:,i] *= -1
    print(Q)
        
    return(lam)

def have_complex_lambda(A, eps, i):
        Q, R = get_QR(A)
        A1 = np.dot(R,Q)
        lambda1 = solve_equation(A, i)
        lambda2 = solve_equation(A1, i)
        if np.all(abs(lambda1 - lambda2) <= eps) and isinstance(lambda1[0], complex) and isinstance(lambda2[0], complex):
            return True
        else:
            return False

def get_eig(A,eps,i,k):
    while True:
        Q, R = get_QR(A)
        A = np.dot(R,Q)
        k+=1
        if np.sqrt(np.sum(A[i + 1:, i]*A[i + 1:, i])) <= eps:
            return A[i,i], False, A, k
        elif np.sqrt(np.sum(A[i + 2:, i]*A[i + 2:, i])) <= eps and have_complex_lambda(A, eps, i):
            return solve_equation(A,i), True, A, k
        
def QR(A, eps):
    eigs = []
    i = 0
    k = 0
    c = 0
    size = A[:,1].size
    while i < size:
        eig = get_eig(A,eps,i,k)
        k = eig[3]
        if eig[1]:
            eigs.append(eig[0])
            i+=2
        else:
            eigs.append(eig[0])
            i+=1
        A = eig[2]
    return eigs,k


        
def QR_arnoldi(A, eps):
    eigs = []
    i = 0
    k = 0
    c = 0
    size = A[:,1].size
    _, A = arnoldi_iteration(A,np.ones(len(A)),len(A))
    while i < size:
        eig = get_eig(A,eps,i,k)
        k = eig[3]
        if eig[1]:
            eigs.append(eig[0])
            i+=2
        else:
            eigs.append(eig[0])
            i+=1
        A = eig[2]
    return eigs,k

def get_eigvectors(A,k):
    A_k = A
    Q_k = np.eye(A.shape[1])
    for k in range(k):
        Q, R = qr(A_k)
        Q_k = Q_k.dot(Q)
        A_k = R.dot(Q)
    return Q_k



A = 10*np.random.random((4,4))
B = A.copy()
print("A:\n", A)
eigs,k = QR(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", special_qr(A))
print("A:\n", B)
eigs,k = QR_arnoldi(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", special_qr(A))

A:
 [[7.92891363 1.18759536 6.08102075 0.53004924]
 [4.71108904 1.87255257 4.99678184 0.02297657]
 [7.35935401 5.70349327 2.47068166 3.35807047]
 [8.31760419 5.69274188 2.61728937 6.02040417]]
Количество итераций:
 50
Собственные значения:
 [16.618577788484085, 3.0056531513495695, -2.6096682064810177, 1.453238379614099]
Матрица собственных векторов
[[-0.44813979  0.51636373 -0.3329724   0.64936015]
 [-0.32056848  0.48212374 -0.2996786  -0.75827784]
 [-0.51860894  0.13231124  0.84417018 -0.03025233]
 [-0.65379763 -0.69528335 -0.29444663 -0.04930427]]
Наибольший собственные вектор:
 [16.58898442  2.93015219 -2.67982296  1.45323838]
A:
 [[7.92891363 1.18759536 6.08102075 0.53004924]
 [4.71108904 1.87255257 4.99678184 0.02297657]
 [7.35935401 5.70349327 2.47068166 3.35807047]
 [8.31760419 5.69274188 2.61728937 6.02040417]]
Количество итераций:
 46
Собственные значения:
 [16.635173636631936, 2.8496173373756575, -2.7522807284351134, 1.4532383796146526]
Матрица собственных векторов
[[-0.44813

In [29]:
A = 10*np.random.random((50,50))+5
B = A.copy()
print("A:\n", A)
eigs,k = QR(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print(get_eigvectors(A,k))

print("A:\n", B)
eigs,k = QR_arnoldi(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print(get_eigvectors(A,k))

A:
 [[12.40677991 11.06523902 12.06784231 ... 13.39925583  5.30405639
  10.89855881]
 [ 8.46513296  5.06418644 12.86781139 ... 11.22281189  9.11221343
   9.65067296]
 [11.92069772  5.32146182  9.9412109  ... 10.62041873  6.6050314
   7.88578049]
 ...
 [ 7.17489823 13.76952379  7.16529336 ...  9.78231589 11.68035865
   5.67216216]
 [ 6.83494342  6.19576303  6.6035497  ...  9.13158201  8.43296613
  11.06265105]
 [ 6.4782291   6.93850039 14.83756021 ...  9.2465547   6.86213584
   7.06572439]]


KeyboardInterrupt: 

In [16]:
np.dot(mat,F[:,0])/F[:,0]

array([ 5.75525217, 11.41044038,  6.26722717])

In [83]:
%%time
A = 10*np.random.random((7,7))+5
B = A.copy()
print("A:\n", A)
eigs,k = QR(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", get_eigvectors(A,k)[:,0])

A:
 [[ 7.40281407  8.27914374 10.72581787  6.11248351 12.90595502  8.16106014
  10.96570311]
 [ 7.95514516  9.94094006  5.38376894  6.24575873  6.58667279  5.37197984
  14.62867383]
 [ 7.74458052 14.59480381  7.16341067 13.46895148 11.4071927   8.88760053
  12.24061186]
 [ 8.44592807 10.83166471 13.6848145  12.771245   13.45023206 12.64500518
   5.7394579 ]
 [10.46435433 12.05519126  5.77216853  7.16893474 13.669194   13.24598462
  10.322806  ]
 [11.45056922 12.83156828 14.33177229 10.99960398  6.26589953 12.9090986
   5.18240826]
 [12.92205416 13.59530909  7.93970526  5.94682779 11.76930663 10.47368775
   7.35520482]]
Количество итераций:
 31
Собственные значения:
 [69.59937828397342, 8.402614584294078, -5.818650827226725, array([1.5435267+4.61181774j, 1.5435267-4.61181774j]), array([-2.03550169+2.04056018j, -2.03550169-2.04056018j])]
Наибольший собственные вектор:
 [-0.34907233 -0.29693914 -0.40442244 -0.42390584 -0.38817697 -0.39895905
 -0.36996257]
Wall time: 15 ms


In [68]:
%%time
print("A:\n", B)
eigs,k = QR_arnoldi(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", get_eigvectors(A,k)[:,0])

A:
 [[13.35706205  7.53886042 14.28980853  7.12985633  9.70496521  6.79385841
  10.73783776]
 [ 5.54727917  9.40253279  8.13685633 13.79883253 12.6217821   8.29121793
  11.95487998]
 [14.4799896   8.40168138  9.89108886  7.09544624 13.14510152 11.32873348
   5.78792737]
 [ 5.5907856   9.61656094  7.49728912  5.84605454 13.20307864 13.66872108
  11.83067892]
 [ 9.96335357  7.26022014  5.91745379  5.00084691  8.5460817  10.71353009
  13.80229811]
 [ 8.57402786  6.56114798 14.11421971 12.26448314 12.90039806  6.25406176
  13.17045603]
 [14.86943708  9.84276166  9.72761468  6.09257472 10.32087455  8.79892079
   6.53662349]]
Количество итераций:
 19
Собственные значения:
 [68.07399575433672, -10.191026997457477, 6.788550300464315, -3.9721666538358473, array([0.59275397+2.85232198j, 0.59275397-2.85232198j]), -2.037890430152606]
Наибольший собственные вектор:
 [-0.38554532 -0.38311707 -0.38889821 -0.37176961 -0.33944084 -0.40584638
 -0.36768231]
Wall time: 13 ms


In [57]:
%%time
A = 10*np.random.random((50,50))+5
B = A.copy()
print("A:\n", A)
eigs,k = QR(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", get_eigvectors(A,k)[:,0])

A:
 [[10.72359075 11.13393155  5.45011535 ...  5.6824868  11.99036914
  14.87507716]
 [ 5.90310659 11.80042802 11.16954808 ...  7.40854133  5.68148828
  10.83780643]
 [11.39337208 12.24684578 14.25807987 ...  5.36864195 14.512801
  14.63870507]
 ...
 [ 5.22989387 10.02020103 14.24603303 ... 10.03507778  5.27834598
   8.61464961]
 [13.21367351  8.10398975 12.7823375  ...  9.29827281  6.62171877
   9.45570276]
 [10.82619811  5.80370336 10.74201628 ... 12.60582502 10.94879985
  10.22128505]]
Количество итераций:
 10990
Собственные значения:
 [498.72786609345746, array([7.63767745+17.71374686j, 7.63767745-17.71374686j]), array([-17.89997941+6.86955109j, -17.89997941-6.86955109j]), array([-18.62898328+4.30689305j, -18.62898328-4.30689305j]), array([-1.2200838+19.00424042j, -1.2200838-19.00424042j]), array([-6.95378503+17.68582648j, -6.95378503-17.68582648j]), array([17.95479879+4.48567692j, 17.95479879-4.48567692j]), -17.9183383697109, array([13.86795686+11.1270776j, 13.86795686-11.1270776j

In [58]:
%%time
print("A:\n", B)
eigs,k = QR_arnoldi(A,0.1)
print("Количество итераций:\n", k)
print("Собственные значения:\n", eigs)
print("Наибольший собственные вектор:\n", get_eigvectors(A,k)[:,0])

A:
 [[10.72359075 11.13393155  5.45011535 ...  5.6824868  11.99036914
  14.87507716]
 [ 5.90310659 11.80042802 11.16954808 ...  7.40854133  5.68148828
  10.83780643]
 [11.39337208 12.24684578 14.25807987 ...  5.36864195 14.512801
  14.63870507]
 ...
 [ 5.22989387 10.02020103 14.24603303 ... 10.03507778  5.27834598
   8.61464961]
 [13.21367351  8.10398975 12.7823375  ...  9.29827281  6.62171877
   9.45570276]
 [10.82619811  5.80370336 10.74201628 ... 12.60582502 10.94879985
  10.22128505]]
Количество итераций:
 1513
Собственные значения:
 [498.7280282768505, array([-2.7002897+12.9878644j, -2.7002897-12.9878644j]), array([-16.12576311+6.55239754j, -16.12576311-6.55239754j]), array([-1.60766762+18.70783326j, -1.60766762-18.70783326j]), array([-13.11151793+5.49223882j, -13.11151793-5.49223882j]), array([-10.87351684+13.19716456j, -10.87351684-13.19716456j]), array([17.95478922+4.48567637j, 17.95478922-4.48567637j]), -17.93096808189397, array([13.86165705+11.12448465j, 13.86165705-11.124484

In [81]:
np.linalg.eig(B)

(array([23.47640262, -4.83968753,  2.07088783,  4.27323568]),
 array([[-0.44312841, -0.24478211,  0.37592954, -0.06644295],
        [-0.50083214, -0.81856322, -0.77630924,  0.36356532],
        [-0.5604678 ,  0.50125584,  0.24344863, -0.59015148],
        [-0.48854911,  0.13703487, -0.44356929,  0.71772336]]))